In [1]:
import zipfile
import olefile

In [2]:
file = 'examples/with_attachments_3.docx'

In [3]:
document = zipfile.ZipFile(file)

In [4]:
document.namelist()

['[Content_Types].xml',
 '_rels/.rels',
 'word/_rels/document.xml.rels',
 'word/document.xml',
 'word/media/image1.emf',
 'word/embeddings/_____Microsoft_Excel______________________.xlsm',
 'word/media/image16.emf',
 'word/media/image15.gif',
 'word/embeddings/oleObject2.bin',
 'word/media/image17.emf',
 'word/embeddings/_____Microsoft_Excel.xlsx',
 'word/media/image18.emf',
 'word/theme/theme1.xml',
 'word/embeddings/_________Microsoft_Visio.vsdx',
 'word/media/image19.emf',
 'word/embeddings/_________Microsoft_Word1.docx',
 'word/media/image13.emf',
 'word/media/image14.png',
 'word/media/image12.emf',
 'word/media/image5.jpeg',
 'word/embeddings/oleObject1.bin',
 'word/media/image3.jpeg',
 'word/media/image2.jpeg',
 'word/embeddings/_________Microsoft_Word.docx',
 'word/media/image6.jpeg',
 'word/media/image4.jpeg',
 'word/media/image8.jpeg',
 'word/media/image7.jpeg',
 'word/media/image11.jpeg',
 'word/media/image9.jpeg',
 'word/media/image10.jpeg',
 'word/settings.xml',
 'word/sty

In [5]:
f = document.open('word/embeddings/oleObject1.bin')
ole = olefile.OleFileIO(f)
ole.listdir()

[['\x01CompObj'], ['\x01Ole10Native'], ['\x03EPRINT'], ['\x03ObjInfo']]

In [7]:
ole.openstream('\x01Ole10Native').read()[:200]

b'\xb4\x1d\x00\x00\x02\x00eiler.json\x00C:\\Users\\dronp\\Desktop\\Programforyou\\mironov\\eiler.json\x00\x00\x00\x03\x00-\x00\x00\x00C:\\Users\\dronp\\AppData\\Local\\Temp\\eiler.json\x00P\x1c\x00\x00{"blocks":[{"x":960,"y":10,"text":"dy(x)","width":100,"height":40,"typ'

### Extract contents from olefile

https://stackoverflow.com/questions/43154439/python-access-embedded-ole-from-office-excel-document-without-clipboard/43409617

In [8]:
stream = ole.openstream('\x01Ole10Native').read()
# original filename in ANSI starts at byte 7 and is null terminated
stream = stream[6:]
filename = ""
for ord_chr in stream:
    if ord_chr == 0:
        break
    filename += chr(ord_chr)
stream = stream[len(filename) + 1:]
filepath = ""
# original filepath in ANSI is next and is null terminated
for ord_chr in stream:
    if ord_chr == 0:
        break
    filepath += chr(ord_chr)
# next 4 bytes is unused
stream = stream[len(filepath) + 1 + 4:]

In [9]:
filename, filepath

('eiler.json', 'C:\\Users\\dronp\\Desktop\\Programforyou\\mironov\\eiler.json')

In [10]:
# size of the temporary file path in ANSI in little endian
temporary_filepath_size = 0
temporary_filepath_size |= stream[0] << 0
temporary_filepath_size |= stream[1] << 8
temporary_filepath_size |= stream[2] << 16
temporary_filepath_size |= stream[3] << 24
temporary_filepath_size

45

In [11]:
stream = stream[4:]
temporary_filepath = ""
for i in range(temporary_filepath_size - 1):
    temporary_filepath += chr(stream[i])
stream = stream[temporary_filepath_size:]
temporary_filepath

'C:\\Users\\dronp\\AppData\\Local\\Temp\\eiler.json'

In [12]:
size = 0 # size of the contents in little endian
size |= stream[0] << 0
size |= stream[1] << 8
size |= stream[2] << 16
size |= stream[3] << 24
stream = stream[4:]
contents = stream[:size] # contents
contents

b'{"blocks":[{"x":960,"y":10,"text":"dy(x)","width":100,"height":40,"type":"\xd0\x9f\xd0\xbe\xd0\xb4\xd0\xbf\xd1\x80\xd0\xbe\xd0\xb3\xd1\x80\xd0\xb0\xd0\xbc\xd0\xbc\xd0\xb0","isMenuBlock":false,"fontSize":14,"textHeight":14,"isBold":false,"isItalic":false},{"x":960,"y":70,"text":"\xd0\xb2\xd0\xb5\xd1\x80\xd0\xbd\xd1\x83\xd1\x82\xd1\x8c x","width":100,"height":30,"type":"\xd0\x9d\xd0\xb0\xd1\x87\xd0\xb0\xd0\xbb\xd0\xbe / \xd0\xba\xd0\xbe\xd0\xbd\xd0\xb5\xd1\x86","isMenuBlock":false,"fontSize":14,"textHeight":14,"isBold":false,"isItalic":false},{"x":960,"y":170,"text":"dx(y, x, t, a1, a2, a3, a4, a5)","width":190,"height":40,"type":"\xd0\x9f\xd0\xbe\xd0\xb4\xd0\xbf\xd1\x80\xd0\xbe\xd0\xb3\xd1\x80\xd0\xb0\xd0\xbc\xd0\xbc\xd0\xb0","isMenuBlock":false,"fontSize":14,"textHeight":14,"isBold":false,"isItalic":false},{"x":960,"y":230,"text":"\xd0\xb2\xd0\xb5\xd1\x80\xd0\xbd\xd1\x83\xd1\x82\xd1\x8c (a4 + a5 * t - a3 * y - a2 * x) / a1","width":260,"height":30,"type":"\xd0\x9d\xd0\xb0\xd1\x87\xd0

In [13]:
def parse_ole_contents(stream):
    # original filename in ANSI starts at byte 7 and is null terminated
    stream = stream[6:]
    filename = ""
    for ord_chr in stream:
        if ord_chr == 0:
            break
        filename += chr(ord_chr)
    stream = stream[len(filename) + 1:]
    filepath = ""
    # original filepath in ANSI is next and is null terminated
    for ord_chr in stream:
        if ord_chr == 0:
            break
        filepath += chr(ord_chr)
    # next 4 bytes is unused
    stream = stream[len(filepath) + 1 + 4:]
    # size of the temporary file path in ANSI in little endian
    temporary_filepath_size = 0
    temporary_filepath_size |= stream[0] << 0
    temporary_filepath_size |= stream[1] << 8
    temporary_filepath_size |= stream[2] << 16
    temporary_filepath_size |= stream[3] << 24
    stream = stream[4:]
    temporary_filepath = ""
    for i in range(temporary_filepath_size - 1):
        temporary_filepath += chr(stream[i])
    stream = stream[temporary_filepath_size:]
    size = 0 # size of the contents in little endian
    size |= stream[0] << 0
    size |= stream[1] << 8
    size |= stream[2] << 16
    size |= stream[3] << 24
    stream = stream[4:]
    contents = stream[:size] # contents
    return filename, contents

In [14]:
ole = olefile.OleFileIO(document.open('word/embeddings/oleObject2.bin'))
ole.listdir()

[['\x01CompObj'], ['\x01Ole10Native'], ['\x03EPRINT'], ['\x03ObjInfo']]

In [15]:
parse_ole_contents(ole.openstream('\x01Ole10Native').read())

('test.py', b'')

In [16]:
file = 'examples/with_attachments.docx'
document = zipfile.ZipFile(file)
document.namelist()

['[Content_Types].xml',
 '_rels/.rels',
 'word/_rels/document.xml.rels',
 'word/document.xml',
 'word/media/image1.emf',
 'word/theme/theme1.xml',
 'word/embeddings/oleObject3.bin',
 'word/media/image3.emf',
 'word/media/image2.emf',
 'word/embeddings/oleObject1.bin',
 'word/embeddings/oleObject2.bin',
 'word/settings.xml',
 'word/webSettings.xml',
 'word/fontTable.xml',
 'docProps/core.xml',
 'word/styles.xml',
 'docProps/app.xml']

In [17]:
ole = olefile.OleFileIO(document.open('word/embeddings/oleObject3.bin'))
ole.listdir()

[['\x01CompObj'], ['\x01Ole10Native'], ['\x03EPRINT'], ['\x03ObjInfo']]

In [18]:
parse_ole_contents(ole.openstream('\x01Ole10Native').read())

('lorem.txt',
 b'Adipisicing est non minim aute reprehenderit incididunt magna ad consectetur ad occaecat anim voluptate culpa fugiat excepteur enim qui nulla ut ea ullamco ut reprehenderit ad do veniam magna velit mollit incididunt amet pariatur nulla sit commodo dolor excepteur magna amet est anim deserunt amet et ad consectetur adipisicing magna nisi voluptate laboris sit laborum ut laborum aliqua sint dolor ullamco enim reprehenderit sunt et elit cillum est dolore consectetur ut labore exercitation ullamco anim ullamco ex ea tempor ad cillum ex eu magna dolor ullamco ad anim aliquip aute dolore ut sit excepteur enim reprehenderit exercitation dolore sit aliquip voluptate nulla cillum non reprehenderit officia labore pariatur pariatur anim in aliqua eu in anim deserunt dolor ex aliquip irure laboris consequat cupidatat id in dolor ut aliqua velit cupidatat dolor do ad adipisicing duis voluptate proident sed quis commodo nulla ut esse officia id dolore in irure voluptate excepteur no

### Extract PDF

In [ ]:
import olefile
from zipfile import ZipFile

def extract_pdf(filename):
    with ZipFile(filename, "r") as document:
        entries = []
      # Find files in the word/embeddings folder of the ZIP file
        for entry in document.namelist():
            if not entry.startswith("word/embeddings/"):
                continue
            else:
                entries.append(entry)
        if not entries:
            return
        # Try to open the embedded OLE file
        for entry in entries:
            with document.open(entry) as f:
                if not olefile.isOleFile(f):
                    continue

                ole = olefile.OleFileIO(f)

                # CLSID for Adobe Acrobat Document
                if ole.root.clsid == "B801CA65-A1FC-11D0-85AD-444553540000":
                    if not ole.exists("CONTENTS"):
                        continue
                    # Extract the PDF from the OLE file
                    data = ole.openstream('CONTENTS').read()
                    # Does the embedded file have a %PDF- header?
                    if data[0:5] == b'%PDF-':
                        data_filename = "examples/tmp/out.pdf"
                else:
                    continue

                # Save the PDF
                with open(data_filename, "wb") as output_file:
                    output_file.write(data)


filename = 'examples/tmp/with_attachments.docx'
extract_pdf(filename)